<a href="https://colab.research.google.com/github/royarkaofficial/DataOps-Lab11-1st-Jan-2025-Post-ML-Data-testing/blob/main/Copy_of_Lab11__Post_ML_Data_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obejctive

The purpose of this lab is to provide hands-on experience in monitoring and evaluating the prformace of a ML model after the deployment. It cobers key ascpets such as : a.Monitoring the data drift b.Evaluating the model performance using standrad metrics c. Detecting when the model has to be retrained

Pre-requisites : Python 3.x
Libraries :scikit-learn , pandas , evidently , matplotlib , numpy

In [ ]:
!pip install mlflow scikit-learn pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00


Step 1 : Simluate the Data amd train a simple Model

In this step , We'll generate synthetic data, train a classification model and log the initial model performance using MLflow

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#Setup MLflow expriment
mlflow.set_experiment("Post-ML Data Testing")

#Generate Syntheticdata
from sklearn.datasets import make_classification
X , y = make_classification(n_samples=1000, n_features=10, random_state=42)

#Split the data into training and testing set
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.3 , random_state=42) #Train - 70% , Test - 30%

#Train the Random Forest model
model = RandomForestClassifier(random_state=42)

#Start the MLflow run
with mlflow.start_run(run_name="Initial_Model_Training"):
  model.fit(X_train , y_train)
  y_pred = model.predict(X_test)

  #Log the metrics
  accuracy = accuracy_score(y_test , y_pred)
  mlflow.log_metric("accuracy" , accuracy)

  #Log the model
  mlflow.sklearn.log_model(model , "random_forest_model")

  #Print performace
  print(f"Initial Model Accuracy : {accuracy:.4f}")
  print("Classification Report : \n" , classification_report(y_test , y_pred))

2025/01/02 10:57:21 INFO mlflow.tracking.fluent: Experiment with name 'Post-ML Data Testing' does not exist. Creating a new experiment.
2025/01/02 10:57:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Initial Model Accuracy : 0.8867
Classification Report : 
               precision    recall  f1-score   support

           0       0.83      0.93      0.88       135
           1       0.94      0.85      0.89       165

    accuracy                           0.89       300
   macro avg       0.89      0.89      0.89       300
weighted avg       0.89      0.89      0.89       300



Step 2: Simulate Data drift
We simulate the data drift by adding noise to the test data , we log the performace of the model on the drifted data

In [ ]:
#Simluate data drift by adding noise to the data
np.random.seed(42) #Reproducible
X_test_drifted = X_test +np.random.normal(0 , 0.5 , X_test.shape)

#Make predictions on Drifted Data
y_pred_drifted = model.predict(X_test_drifted)

#Evaluate the performance on Drifted Data
accuracy_drifted = accuracy_score(y_test , y_pred_drifted)
print(f"Model Accuracy on Drifted Data : {accuracy_drifted:.4f}")
print("Classification Report on Drifted Data : \n" , classification_report(y_test , y_pred_drifted))

Model Accuracy on Drifted Data : 0.7867
Classification Report on Drifted Data : 
               precision    recall  f1-score   support

           0       0.79      0.72      0.75       135
           1       0.79      0.84      0.81       165

    accuracy                           0.79       300
   macro avg       0.79      0.78      0.78       300
weighted avg       0.79      0.79      0.79       300



Step 3 : Monitor the data and model performance using ML flow

In [ ]:
#Define the HELPER function to log performance metrics
def log_classification_metrics(y_true , y_pred , run_name):
  with mlflow.start_run(run_name=run_name):
    accuracy = accuracy_score(y_true , y_pred)
    mlflow.log_metric("accuracy" , accuracy)

    #Other metrics can be logged , like precision , recall and F1-score
    report = classification_report(y_true , y_pred , output_dict=True)
    mlflow.log_metric("precision",report['weighted avg']['precision'])
    mlflow.log_metric("recall" , report['weighted avg']['recall'])
    mlflow.log_metric("f1-score" , report['weighted avg']['f1-score'])

#Log initial model mtrics
log_classification_metrics(y_test , y_pred , "Initial_Model_Metrics")

#Log metrics on drifted data
log_classification_metrics(y_test , y_pred_drifted , "Performance_after_data_drift")

Step 4 : Model Retraining based on Drift

Based on the performace degradation due to data drift , you can decide whether to retrain the model. If performace drops below a threshold (e.g Class 0 - 5%), retrain  the model

In [ ]:
#Decide to retrain if the accuracy drops significantly
if accuracy_drifted < accuracy - 0.05 :
  print("Signicant performance drop detected , Retraining the model...")

  #Retrain the model
  with mlflow.start_run(run_name = "Model_Retraining"):
    model.fit(X_train , y_train)
    y_pred_retrained = model.predict(X_test_drifted)
    new_accuracy = accuracy_score(y_test , y_pred_retrained)

    #log the retrained model
    mlflow.sklearn.log_model(model , "retrained_random_forest_model")
    mlflow.log_metric("new_accuracy" , new_accuracy)

    print(f"New model accuracy after retraining : {new_accuracy:.4f}")
else:
  print("Model performance is stable.No re-training required")


Signicant performance drop detected , Retraining the model...


2025/01/02 11:01:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


New model accuracy after retraining : 0.7867
